<a href="https://colab.research.google.com/github/julialromero/regional-representations-graph-model/blob/master/dataset/earth_engine/download_landsat_imagery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
! pip install pydrive

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.4/987.4 kB 8.1 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 1.2 MB/s eta 0:00:00ta 0:00:01
  Created wheel for pydrive: filename=PyDrive-1.3.1-py3-none-any.whl size=27436 sha256=91449264e836ba8bd6cdcbe6e30b4ab632b0bd3b00e34f5152c6fa83f527761e
  Stored in directory: /home/juro4948/.cache/pip/wheels/1e/99/07/235c248b6e051812fc5b695a7506b9f309fc0bbb4223ec2296
Successfully built pydrive


In [1]:
! conda env list


# conda environments:
#
base                     /home/juro4948/miniconda3
myenv                 *  /home/juro4948/miniconda3/envs/myenv



In [2]:
import os
print (os.environ['CONDA_DEFAULT_ENV'])

myenv


In [3]:
import os
import ee
import geemap
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd

In [4]:
import ee
# ee.Authenticate()
ee.Initialize()

In [5]:
tract_data_dir = r'../Tracts/nyc_metro_boundaries/nyc_metro_boundaries.shp'
temp_tract_data_dir = "../Tracts/nyc_metro_boundaries/temp.shp"

df = gpd.read_file(tract_data_dir)
df.rename(columns={'GEOID_TRAC': 'GEOID'}, inplace=True)  
# print(df.head())
shapefile = df[['GEOID', 'geometry']] 

In [6]:
def get_img(geoid):
    feat = tracts.filter(ee.Filter.eq('GEOID', geoid)).first()
    roi = feat.geometry()

    collection = (
      ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA')  # https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1
      .filterBounds(roi)
      .filter(ee.Filter.contains('.geo', roi))
      .filterDate('2020-01-01', '2023-01-01')
      .sort('CLOUD_COVER')
    )
    top = collection.limit(10)

    # take the median pixel value; clip to the census geometry 
    # composite = top.median().clip(roi)
    img = top.first().clip(roi)
    return img#, geoid
    

def createDir(dirPath):
  try:
    os.makedirs(dirPath)
  except:
    pass

def init_dir(subdir):
  if os.path.exists(subdir):
      if 'least_cloudy_clipped_highres.tif' in os.listdir(subdir):
        print('pass')
        return True
      else:
        print(f'Redoing: {subdir}')

  if not os.path.exists(subdir):
      print('creating directory')
      createDir(subdir)
      return True

In [8]:
# Map = geemap.Map()
# Map.setCenter(-74.0060, 40.7128, 8)
# visParams = {'bands': ['B6'], 'min': 0, 'max': 0.2}
# Map.addLayer(top, visParams, 'tracts')
# Map

In [9]:
# for interval in range(0, shapefile.shape[0], 20):
#     geoid = shapefile.iloc[interval+i]['GEOID']
#     print(geoid)

In [10]:

save_dir = 'download_landsat_images'
for interval in range(0, shapefile.shape[0], 20):
    try:
        sub = shapefile.iloc[interval:interval+20]
        sub.to_file(temp_tract_data_dir)
    except:
        sub = shapefile.iloc[interval:-1]
        sub.to_file(temp_tract_data_dir)
    tracts = geemap.shp_to_ee(temp_tract_data_dir)

    geoids = tracts.aggregate_array('GEOID')
    result = geoids.map(get_img)
    for i in range(0, sub.shape[0]):
#         print('-')
        geoid = shapefile.iloc[interval+i]['GEOID']
        
        subdir = f"{save_dir}/{geoid}"
        filename = f"{subdir}/least_cloudy_clipped_highres.tif"
        if os.path.exists(filename):
#             print(f'continue: {filename}')
            continue
        init_dir(subdir)

        img = ee.Image(result.get(i))
        
        if geoid == '36071015203':
            print('TARGET')


        geemap.ee_export_image(
                img, filename=filename, file_per_band=False
            )
        if not os.path.exists(filename):
            img = img.select(['B1', 'B2', 'B3','B4', 'B5', 'B6', 'B7', 'B8'])
            geemap.ee_export_image(
                    img, filename=filename, file_per_band=False
                )
            
        if not os.path.exists(filename):
            img = img.select(['B1', 'B2', 'B3','B4', 'B5', 'B6', 'B7'])
            geemap.ee_export_image(
                    img, filename=filename, file_per_band=False
                )
        if not os.path.exists(filename):
            img = img.select(['B1', 'B2', 'B3','B4', 'B5', 'B6'])
            geemap.ee_export_image(
                    img, filename=filename, file_per_band=False
                )
            
        if not os.path.exists(filename):
            img = img.select(['B1', 'B2', 'B3','B4', 'B5'])
            geemap.ee_export_image(
                    img, filename=filename, file_per_band=False
                )

        
        if os.path.exists(filename): 
            print('saved')
      

    print(f'-----DONE {interval} - {interval + sub.shape[0]}')
    

-----DONE 0 - 20
-----DONE 20 - 40
-----DONE 40 - 60
-----DONE 60 - 80
-----DONE 80 - 100
-----DONE 100 - 120
-----DONE 120 - 140
-----DONE 140 - 160
-----DONE 160 - 180
-----DONE 180 - 200
-----DONE 200 - 220
-----DONE 220 - 240
-----DONE 240 - 260
-----DONE 260 - 280
-----DONE 280 - 300
-----DONE 300 - 320
-----DONE 320 - 340
-----DONE 340 - 360
-----DONE 360 - 380
-----DONE 380 - 400
-----DONE 400 - 420
-----DONE 420 - 440
-----DONE 440 - 460
-----DONE 460 - 480
-----DONE 480 - 500
-----DONE 500 - 520
-----DONE 520 - 540
-----DONE 540 - 560
-----DONE 560 - 580
-----DONE 580 - 600
-----DONE 600 - 620
-----DONE 620 - 640
-----DONE 640 - 660
-----DONE 660 - 680
-----DONE 680 - 700
-----DONE 700 - 720
-----DONE 720 - 740
-----DONE 740 - 760
-----DONE 760 - 780
-----DONE 780 - 800
-----DONE 800 - 820
-----DONE 820 - 840
-----DONE 840 - 860
-----DONE 860 - 880
-----DONE 880 - 900
-----DONE 900 - 920
-----DONE 920 - 940
-----DONE 940 - 960
-----DONE 960 - 980
-----DONE 980 - 1000
-----DONE

KeyboardInterrupt: 

In [69]:
def get_regions_to_retry():
    l = []
    counter=0
    for root, dirs, files in os.walk("download_landsat_images/"):
        if '.ipynb_checkpoints' in root:
            continue
        for f in files:
            if f == 'least_cloudy_clipped_highres.tif':
                counter+=1
        if 'least_cloudy_clipped_highres.tif' not in files:
            l.append(root)
#             print(root)
            
    l.remove('download_landsat_images/')
    al = len(os.listdir("download_landsat_images/"))
    print(f'Num folders total: {al}')
    print(f'Num target files total: {counter}')
    print()
    l = [item[-11:] for item in l]
    return l
    
r = get_regions_to_retry()

for geoid in r:
    shapefile.loc[shapefile.GEOID == geoid].to_file(temp_tract_data_dir)
    tracts = geemap.shp_to_ee(temp_tract_data_dir)
    
    i = get_img(geoid)
    save_dir = 'download_landsat_images'
    subdir = f"{save_dir}/{geoid}"
    filename = f"{subdir}/least_cloudy_clipped_highres.tif"

    geemap.ee_export_image(
            i, filename=filename, file_per_band=False
        )
    
    if not os.path.exists(filename):
            img = img.select(['B1', 'B2', 'B3','B4', 'B5', 'B6', 'B7'])
            geemap.ee_export_image(
                    img, filename=filename, file_per_band=False
                )
    if not os.path.exists(filename):
        img = img.select(['B1', 'B2', 'B3','B4', 'B5', 'B6'])
        geemap.ee_export_image(
                img, filename=filename, file_per_band=False
            )
    if not os.path.exists(filename):
        img = img.select(['B1', 'B2', 'B3','B4', 'B5'])
        geemap.ee_export_image(
                img, filename=filename, file_per_band=False
            )
            
            
    print('--\n')

Num folders total: 8232
Num target files total: 8223

Generating URL ...
An error occurred while downloading.
Total request size (189236294 bytes) must be less than or equal to 50331648 bytes.
Generating URL ...
An error occurred while downloading.
Request payload size exceeds the limit: 10485760 bytes.
Generating URL ...
An error occurred while downloading.
Request payload size exceeds the limit: 10485760 bytes.
Generating URL ...
An error occurred while downloading.
Request payload size exceeds the limit: 10485760 bytes.
--

Generating URL ...
An error occurred while downloading.
Total request size (53337888 bytes) must be less than or equal to 50331648 bytes.
Generating URL ...
An error occurred while downloading.
Request payload size exceeds the limit: 10485760 bytes.
Generating URL ...
An error occurred while downloading.
Request payload size exceeds the limit: 10485760 bytes.
Generating URL ...
An error occurred while downloading.
Request payload size exceeds the limit: 10485760 

In [7]:
# # plot all
# Map = geemap.Map()
# Map.setCenter(-74.0060, 40.7128, 8)
# visParams = {'bands': ['B6'], 'min': 0, 'max': 0.2}
# Map.addLayer(collection.first(), visParams, 'Landsat8')
# Map

In [1]:
img

NameError: name 'img' is not defined

In [236]:
import tqdm


stats = pd.DataFrame(columns = [
    'geoid',
    'max',
 'mean',
 'min',
 'sample_sd',
 'sample_var',
 'sum',
 'sum_sq',
 'total_count',
 'total_sd',
 'total_var',
 'valid_count',
 'weight_sum',
 'weighted_sum'])

img_list = ee.List([])


def get_img(geoid):
    feat = tracts.filter(ee.Filter.eq('GEOID', geoid)).first()
    roi = feat.geometry()

    collection = (
      ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA')  # https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1
      .filterBounds(roi)
      .filter(ee.Filter.contains('.geo', roi))
      .filterDate('2020-01-01', '2023-01-01')
      .sort('CLOUD_COVER')
    )
    top = collection.limit(10)

    # take the median pixel value; clip to the census geometry 
    # composite = top.median().clip(roi)
    img = top.first()
    return img#, geoid
    
def save_images(geoid):
    print(geoid_list)
    for geoid in geoid_list:
        print(geoid)
    save_dir = 'download_landsat_images/'
    subdir = f"{save_dir}{geoid}"


    if os.path.exists(subdir):
        if len(os.listdir(subdir)) >= 10:
#               print('pass')
          return True
        else:
#               print(f'Redoing: {subdir}')
            pass

    if not os.path.exists(subdir):
#             print('creating directory')
        createDir(subdir)

    
    geemap.ee_export_image(img, filename=f'{subdir}/least_CC.tif', region=roi, scale=90)

    # for image collection stats 
#         temp = pd.DataFrame(data=img.toArray().getInfo()).T
#         temp.columns=temp.keys().getInfo()
#         temp['geoid'] = geoid

#         stats = pd.concat([stats, temp], axis=0)
#         stats.to_csv('save_dir/_CC_stats.csv', index=False)


#         for img_idx in top.aggregate_array('system:index').getInfo():
#             img = ee.Image('LANDSAT/LC08/C02/T1_TOA/' + img_idx)
#             filename = f"{subdir}/{img_idx}.tif"

#             new_img = img.clip(roi)

#             geemap.ee_export_image(
#                 new_img, filename=filename, scale=90, region=roi, file_per_band=False
#             )

    return 

In [59]:
# plot all
Map = geemap.Map()
Map.setCenter(-74.0060, 40.7128, 8)
visParams = {'bands': ['B6'], 'min': 0, 'max': 0.2}
Map.addLayer(collection.first(), visParams, 'Landsat8')
# Map.addLayer(roi, {}, 'Census Tracts')
Map

Map(center=[40.7128, -74.006], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chi…

In [ ]:
Map = geemap.Map()
Map.setCenter(-74.0060, 40.7128, 8)
visParams = {'bands': ['B6'], 'min': 0, 'max': 0.2}
Map.addLayer(new_img, visParams, 'Landsat8')
# Map.addLayer(roi, {}, 'Census Tracts')
Map

In [10]:
save_dir = 'download_landsat_images/'
def Count_files_in_subd():
    l = []
    
    for root, dirs, files in os.walk(save_dir):
        # print("{} in {}".format(len(files), root))
        l.append(len(files))
#         if len(files) == 0:
#           print(dirs)
    al = len(os.listdir(save_dir))
    print(f'Num folders total: {al}')
    print(f'Num folders with 0 files: {l.count(0)}')
    print(f'Num folders with >0 files: {al - l.count(0)}')
    print()
Count_files_in_subd()

Num folders total: 402
Num folders with 0 files: 3
Num folders with >0 files: 399



## Part 2: Define a dataset object

In [ ]:
 ! pip install torchgeo

In [113]:
import torch

In [115]:
import torchgeo
from torch.utils.data import DataLoader
from torchgeo.datasets import RasterDataset, stack_samples, unbind_samples
from torchgeo.datasets.utils import download_url
from torchgeo.samplers import RandomGeoSampler

In [116]:
dir_path = 'content/drive/MyDrive/landsat/download_landsat_images/test1/09003514500_0.tif'
class Landsat8(RasterDataset):
    filename_glob = "T*_B02_10m.tif"
    filename_regex = r"^.{6}_(?P<date>\d{8}T\d{6})_(?P<band>B0[\d])"
    date_format = "%Y%m%dT%H%M%S"
    is_image = True
    separate_files = True
    all_bands = ["B02", "B03", "B04", "B08"]
    rgb_bands = ["B04", "B03", "B02"]

    def plot(self, sample):
        # Find the correct band index order
        rgb_indices = []
        for band in self.rgb_bands:
            rgb_indices.append(self.all_bands.index(band))

        # Reorder and rescale the image
        image = sample["image"][rgb_indices].permute(1, 2, 0)
        image = torch.clamp(image / 10000, min=0, max=1).numpy()

        # Plot the image
        fig, ax = plt.subplots()
        ax.imshow(image)

        return fig

SyntaxError: ignored